---
## Thursday 1/3/2018:
Two important additions to the 'cociprocess' script:
* A new heuristic to handle the negative time span: in case the timespan calculated with the cited document is negative, we calculate its timespan with the date in the ‘year’ field, and in case this one is positive we take this value instead.
* Update the dates dictionary in case we find a much detailed date value for the same id

Other issues have been fixed in the script that will execute in parallel cociprocess.py. Most importantly, the script will keep track of the processes in execution and re-run a process in case it stops for some reason.  
I have executed the script on our server for the ‘open/’ data stored. I will check the results of it tomorrow. 

---
## Friday 2/3/2018:
I had run the script father that creates N number of subprocesses which processes the crossref dumb and create the resulting CSV files. N equals the number of inner folders in the root directory path given to the father script. Here is the actual command (executed from the ‘script/’ folder):

```nohup python3.5 multi_cociprocess.py -script cociprocess.py -in /srv/data/coci/open/ -out /srv/data/coci/results/open/ -lookup /home/ivanhb/lookup.csv &```

The first output raised some small issues, which I have fixed with additional controls added to the script. 
Despite this there is a much more concerning problem: Although the processes have been all created they stay too much time in sleeping status. The processes don’t run continuously, this behavior slows down a lot the expected performances.  
I have tried several tests, and modified several parameters of the Popen python command, but still without positive feedbacks.

---
## Monday 5/3/2018:
We are still stuck with a processes parallelism problem. Both me and Silvio are looking for techniques and possible solutions to this problem. Since we still cannot let all the processes stay active, their scheduling paradigm slows a lot the entire dump elaboration. I spent the day searching the web for some approaches and strategies that might change this situation. 
* Modifying the multi_cociprocess.py script by using the multiprocessing library. 
* Applying different parameters to the Popen call that could affect its execution.
* Monitoring the outputs of the scripts, in case unexpected behaviors happened  
* Creating multiple cociprocess.py scripts to execute for each different process

Unfortunately still without positive feedbacks from the execution.

---
## Tuesday 6/3/2018:  
After a quick review of the situation and the processes parallelism problem, together with Silvio we decided to give another last chance, and try one last approach that might help us solve our problem.  
We have decided to use [Parallel Gnu](https://www.gnu.org/software/parallel/) and create a small script that generates a file containing all the commands that need to be executed (a cociprocess.py call for each folder of the crossref dump). The script output will be used as an input to Parallel Gnu.  
Even this last strategy, and after the application of different parameters to parallel, didn't give us positive results.  
Therefore we decided to move on, and elaborate an alternative strategy to process our crossref dump. The main idea is to limit as much as possible the GET requests (we will use only direct DOI requests).  
I have created a first python script that generates our new global index containing the publication date of each DOI, along with the updated lookup codes list. The second Script needed is the one that will generate all the data and provenance.

---
## Wednesday 14/3/2018:
Today with Silvio we have discussed the overall current situation regarding the coci project and the SAVE-SD adjustments. 
* SAVE-SD: I have listed all the comments made by the reviewers and classified them in points that we agree/not-agree we need to change. And discussed with Silvio all the other doubtful aspects.
* Coci processing: There was a corrupted file in the limited dataset and we have been able to successfully correct it. I have launched the global index generator on the new limited correct dataset. The script have terminated successfully and the total number of entries we have in the dates csv file is equal to: 7759816.

I will now first concentrate on the SAVE-SD paper modifications, before moving back to the coci project.

---
## Thursday 15/3/2018:
I have started and almost finished modifying the SAVE-SD paper of OSCAR. I am planing on working on it tomorrow morning also and send it to Silvio (the .odt draft document).   
All the modifications made have been discussed with Silvio yesterday.   
Tomorrow, I will give a brief look on the adjustments made today and just add 2 screenshots to show the scholarydata and Wikidata examples.

---
## Friday 16/3/2018:
I have completed the last modifications in the OSCAR draft and send it to Silvio. The last things I have done on it are the integrations of new figures and modify the diagram of OSCARs workflow.  
Next I moved back on COCI. The global indexes generated (also for the limited dataset) look correct.  
So I tested the second script, which will process all the references and generate all the data, as it was in our first attempt to process COCI. After some adaptions the results obtained locally are those expected. I have lunched it also on the server for the ‘limited’ directory, by typing the following command from the script directory: 

``` nohup python multi_cociprocess_refs.py -pycmd python -script cociprocess_refs.py -in /srv/data/coci/limited/ -glob /srv/data/coci/results/plan-b/limited/ -out /srv/data/coci/results/plan-b/limited/ &```

---
## Monday 19/3/2018:
At this point we need to prepare the RDF generation phase for the COCI csv data produced. I have started first by modifying a bit the [citation.py](https://github.com/opencitations/coci/blob/master/script/citation.py) module written by Silvio to handle the COCI case:    
The citation object can already be initialized given the Duration and Publication-date as value, this will prevent the citation module from calculating it. The default values are None in case we don’t have these values at the initialization.  
In case the oci is not specified let the object init create it, otherwise (coci case) we already have it, and therefor we can specify it.

In [ ]:
def __init__(self,
                 ...
             ,creation_date = None, duration= None, oci= None):
    
    
    if oci == None:
            self.oci = citing_entity_local_id + "-" + cited_entity_local_id        
    
    ...
    
    self.creation_date = creation_date
    self.duration = duration
    if self.creation_date == None and self.duration == None :

Next I moved on writing [coci_rdfgen.py](https://github.com/opencitations/coci/blob/master/script/coci_rdfgen.py) a python script which will open the COCI csv data files and call the citation.py to generate the RDF entities and generate the corresponding files.

---
## Tuesday 20/3/2018:
I continued working on both the citation.py and coci_rdfgen.py scripts.  
Some url values to apply for the COCI entries are:  
Base URL: https://w3id.org/oc/index/coci/  
Agent URL: https://w3id.org/oc/index/coci/prov/pa/1  
DOI Prefix to add for the ‘citing’ and ‘cited’ values = http://dx.doi.org/  
For the COCI case we will not include all the identifier triples: datacite: … . Along with it’s label value. 
The coci_rdfgen.py works as follow now:
* Takes the root directory of the coci processing results. (e.g: ‘results/’ such that this directory contains ‘results/[[1…N]]/data,prov,index’)
* Calls the Citation script to create a graph for each different entry in the data csv files.
* writes all the results in a single file (could be specified with -out) in ‘NT’ format.

---
## Wednesday 21/3/2018:
We have arranged a DASPLAB meeting in the morning and discussed several aspects related to the RDF dataset generation of the COCI data, and how we will use OSCAR and LUCINDA to visualize the COCI triplestore contents. 
Some important adaptions should be made on LUCINDA: 
* We need to enable the system visualize data from external resources, not only those retrieved from the SPARQL query execution. e.g: we might use Crossref API with a specific DOI and retrieve a specific field from it.
* Create a Graph Network starting from the actual citation (citing node, an edge, and the cited node), and build the adjacent nodes. 

Right now I am focusing on the first point and I have started modifying the LUCINDA code. 

---
## Thursday 22/3/2018:
Some adjustments in OSCAR and LUCINDA:
* When applying these queries:  
(a) http://opencitations.net/search?text=Shotton&rule=author_lname&bc_4=and&text=David&rule=author_fname  
(b) http://opencitations.net/search?text=shotton&rule=author_lname&bc_4=and&text=david&rule=author_fname  
It turns out that (a) gives a wrong number of works, while (b) gives is correct. The problem resides in the heuristics application, since (b) got ‘shotton’ and ‘david’ without capitalizations. 
I have successfully fixed this problem.
* All the OSCAR links (documents and authors) should point to LUCINDA 
* Resource names should not start with a ‘/‘. e.g: /br/1, /ra/1   
* Some labels values:  "Out citations" -> "References" | "In citations" -> “Citations” | "Publication year" -> "Publication date"
* Remove all the Console logs prints
* In case the list of Citations and References is empty then remove it from the menu. **THIS POINT IS STILL NOT FIXED**.
* in case “Cites 0 documents”, remove the entire entry from the metrics section. To make this possible a new field could be specified in the configuration file: ‘respects’. It will contain an array of heuristic functions the value must respect in order to be added.

All these changes have been pushed in the repositories [oscar](https://github.com/opencitations/oscar) and [lucinda](https://github.com/opencitations/lucinda) on the master branch.

A problem in COCI processing came out while working on the ‘limited’ dataset: in case of a NO-DOI ERROR this information has not been pushed correctly in the csv errors. I have fixed this issue and updated the corresponding repository [coci](https://github.com/opencitations/coci)

---
## Friday 23/3/2018:

**LUCINDA:**  
The possibility to use external data usage: now it is possible to make external calls (e.g: API call) and visualize the data retrieved inside LUCINDA. Here is how does this happen:
1) Define a function inside the configuration file:
```
function call_crossref(str_doi, field){
  var call_crossref_api = "https://api.crossref.org/works/";
  var call_url =  call_crossref_api+ encodeURIComponent(str_doi);
  ...
}
```
2) For the wanted category you can specify that some of its data are external and use the previous function through the **ext_data** key:
```
"ext_data": {
            "crossref4doi": {"name": call_crossref, "param": {"fields":["id_lit","FREE-TEXT"],"values":[null,1]}}
          },
```
3) To access the resulting data and visualize it inside the interface we can specify a similar entry:
```
{"fields": ["FREE-TEXT", "EXT_DATA"], "values": ["Publisher: ", "crossref4doi.message.publisher"]}
```

<img src="img/lucinda-api-mod.png" style="display: inline-block; height: 350px; width: 720px;"/>



**OSCAR :**
* No need to put **bc_[[NUMBER]]** tag in the url, instead only **bc** is enough.  
e.g: ...oscar/search.html?text=shotton&rule=author_lname**&bc=**and&text=david&rule=author_fname
* Building a new configuration for the COCI case [search-conf-coci.js](https://github.com/opencitations/oscar/blob/master/static/js/search-conf-coci.js). I have started by defining some possible rules OSCAR needs to detect when looking for OCIs. When having as input the citing doi, e.g:
```
{
      "name":"citingdoi",
      "label": "Citing DOI",
      "advanced": true,
      "freetext": false,
      "heuristics": [[lower_case]],
      "category": "citation",
      "regex":"(10.\\d{4,9}\/[-._;()/:A-Za-z0-9][^\\s]+)",
      "query": [
            "{",
            "?iri cito:hasCitingEntity '[[VAR]]' .",
            "}"
      ]
},
```
The **Data Category** we need OSCAR to generate is **citation**:
```
"name": "citation",
"label": "Citation",
"macro_query": [
      "SELECT DISTINCT ?iri ?oci ?citing_doi ?cited_doi ?creationdate ?timespan",
          "WHERE  {",
            "[[RULE]]",
            "OPTIONAL {",
              "?iri cito:hasCitingEntity ?citing_doi .",
              "?iri cito:hasCitedEntity ?cited_doi .",
              "?iri cito:hasCitationCreationDate ?creationdate .",
              "?iri cito:hasCitationTimeSpan ?timespan .",
            "}",
          "}",
],
```

---
## Monday 26/3/2018:
We need to apply the “External data calls” behavior also for OSCAR.  
In this case for each row of the table we can put an additional column/field, and its value is retrieved through an external function call (e.g: API call on Crossref).  
To extend the columns visualized we need to add an additional entry on the wanted category:

```{"value": "ext_data.crossref4doi.message.publisher", "title": "Publisher", "column_width":"8%","type": "text"}```

An interesting aspect that should be handled is the Asyncro/Syncro execution of the External function. A Syncro call in the case of OSCAR will limit its performance, therefor an Asyncro execution is highly required in this case.

---
## Tuesday 27/3/2018:
I have been able to successfully integrate the “External data calls” behavior inside OSCAR, with the Asyncro/Synco behavior. To explain and show how does this work I used the same Crossref API call used for the LUCINDA case:  

1) Define the EXT_DATA call function inside the category wanted this way, the function specified should be defined in the configuration file (call_crossref):
```
"ext_data": {
        "crossref4doi": {"name": call_crossref, "param": {"fields":["doi"]}, "async": true}
      },
```
 
2) Once OSCAR is loaded the table is generated as normal. The columns that wish to incorporate external data (out of the SPARQL query results), will call the function specified inside their configuration.
```
{"value": "ext_data.crossref4doi.message.publisher", "title": "Publisher", "column_width":"13%", "type": "text"}
```

In case the function is ASYNC, the table will be generated and the data of this column will be gradually populated once the requests are fully elaborated. On the other hand with ASYNC=false the table will not be displayed unless all the requests have done.
Here we have a snapshot of the ASYNC case.

<img src="img/async-oscar.png" style="display: inline-block; height: 350px; width: 720px;"/>

---
## Wednesday 28/3/2018:
Handling the Sort, Filtering, and Linking operations on the “EXT_DATA” columns/fields.  

1) Sort: In this case when applying the sort for a specific field the results are sorted on the values available. Therefore new values that become available after the sort might appear in inappropriate positions (at the end of the table).

2) Filter: as for the sort case the values to filter will gradually appear in the filter section. There is an issue still need a further examination in this case: the arriving of a new value to the table, consequently calls an update of the interface which bring the user to the default interface appearance (e.g: first page).

3) Link: the new columns can associate their values to any link from the fields of the sparql results. In this case the behavior is still the same as for the standard columns.

---
## Thursday 29/3/2018:
I have successfully been able to handle the previous issue of yesterday on point (2). The elements in the interface are directly updated without affecting the other fixed components already in the page.  

The list of values to filter on the left section are now scrollable once the dropdown button for the field is clicked. In the configuration file is still possible to define a maximum number of rows, in case all the rows exceed such number the arrows are added to handle the paging. Note that is possible to set a very high maximum rows val (or remove the limit), in case we don’t want to keep the paging arrows in all cases.  
To distinguish this last integration (the scroll bar), a new branch on the OSCAR repository has been created. I will remove it soon and merge all on the master as soon as I am sure everything looks fine.  

---
## Friday 30/3/2018:
I have merged all the modifications made on OSCAR in one branch (the main one ‘master’). This version is ready to be added on the website of OpenCitaions. I will list all the appropriate steps to Silvio in order to do that.  

I moved back to Lucinda where I am working on another important aspect: Together with Silvio we decided to make the data retrieved from OSCAR async and running in background, once these operations are done the bottom menu will appear and no further calls on OSCAR are made.  
Another point of LUCINDA that needs a further investigation is the “Cited by [[N]] different documents” entry in the metrics section. It seems to me that the number N is not correct on some cases, so I need to check and decide whether to keep it or not.